In [0]:
# -2
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# -1
!wget -O "./drive/My Drive/Colab Notebooks/SigNet/signatures.rar" "http://www.cedar.buffalo.edu/NIJ/data/signatures.rar"

--2019-06-29 17:50:53--  http://www.cedar.buffalo.edu/NIJ/data/signatures.rar
Resolving www.cedar.buffalo.edu (www.cedar.buffalo.edu)... 128.205.237.100
Connecting to www.cedar.buffalo.edu (www.cedar.buffalo.edu)|128.205.237.100|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cedar.buffalo.edu/NIJ/data/signatures.rar [following]
--2019-06-29 17:50:53--  https://cedar.buffalo.edu/NIJ/data/signatures.rar
Resolving cedar.buffalo.edu (cedar.buffalo.edu)... 128.205.237.100
Connecting to cedar.buffalo.edu (cedar.buffalo.edu)|128.205.237.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 253587033 (242M) [text/plain]
Saving to: ‘./drive/My Drive/Colab Notebooks/SigNet/signatures.rar’

./drive/My Drive/Co 100%[===================>] 241.84M  63.3MB/s    in 4.6s    

2019-06-29 17:50:58 (53.0 MB/s) - ‘./drive/My Drive/Colab Notebooks/SigNet/signatures.rar’ saved [253587033/253587033]



In [0]:
# 0
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
from natsort import natsorted, ns

def get_image_list():
  org_signs = os.listdir("drive/My Drive/Colab Notebooks/SigNet/signatures/signatures/full_org")
  forg_signs = os.listdir("drive/My Drive/Colab Notebooks/SigNet/signatures/signatures/full_forg")
  org_signs = [s for s in org_signs if s.endswith(".png")]
  forg_signs = [s for s in forg_signs if s.endswith(".png")]
  org_signs = natsorted(org_signs, alg=ns.IGNORECASE)
  forg_signs = natsorted(forg_signs, alg=ns.IGNORECASE)
  #org_signs,forg_signs = clean_lists(org_signs,forg_signs)
  return org_signs, forg_signs

In [0]:
# data cleaning
def check_lists(org_signs,forg_signs):
  flag = False
  #print(len(org_signs),len(forg_signs))
  for i in range(len(org_signs)):
    org_ext = org_signs[i][8:]
    forg_ext = forg_signs[i][9:]
    if org_ext != forg_ext:
      flag = True
      #print(i,org_ext,forg_ext)
  '''if(flag):
    #print("Mismatches found")
  else:
    #print("No mismatch found")
'''
def refine_lists(org_signs,forg_signs):
  refined_org_signs = []
  for i in range(len(org_signs)):
    if "_41_" in org_signs[i]:
      continue
    refined_org_signs.append(org_signs[i])
  
  refined_forg_signs = []
  for i in range(len(forg_signs)):
    if "_41_" in forg_signs[i]:
      continue
    refined_forg_signs.append(forg_signs[i])
  return refined_org_signs, refined_forg_signs

def get_clean_lists():
  org_signs, forg_signs = get_image_list()
  check_lists(org_signs,forg_signs)
  org_signs, forg_signs = refine_lists(org_signs,forg_signs)
  check_lists(org_signs,forg_signs)
  return org_signs, forg_signs

In [0]:
import random
import pandas as pd
import numpy as np

def get_dataframe(org_signs,forg_signs):
  no_of_ppl = len(org_signs)//24

  raw_data = {"image_1":[], "image_2":[], "label":[]}
  for i in range(no_of_ppl):
    i1_batch_1 = []
    i1_batch_2 = []
    i2_batch = []

    start = i*24
    end = (i+1)*24

    for j in range(start,end): 
      i1_batch_1.append(os.path.join(PATH_ORG,org_signs[j]))
      i1_batch_2.append(os.path.join(PATH_ORG,org_signs[j]))
      raw_data["label"].append(0)

    temp_rot = (i1_batch_1[-12:]+i1_batch_1[:-12])
    i1_batch_1.extend(i1_batch_2)

    for elem in temp_rot:
      i2_batch.append(elem)

    for j in range(start,end): 
      i2_batch.append(os.path.join(PATH_FORG,forg_signs[j]))
      raw_data["label"].append(1)

    raw_data["image_1"].extend(i1_batch_1)
    raw_data["image_2"].extend(i2_batch)
  #print(len(raw_data["image_1"]),len(raw_data["image_2"]),len(raw_data["label"]))
  df = pd.DataFrame(raw_data, columns = ["image_1","image_2","label"])
  df=df.reindex(np.random.permutation(df.index))
  return df

In [0]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

def get_dataset(subset=None):
  org_signs,forg_signs = get_clean_lists()
  df = get_dataframe(org_signs,forg_signs)
  train_set, val_set = train_test_split(df,test_size=0.3,random_state=0)
  if(subset.lower()=="train"):
    dataset = train_set
  else:
    dataset = val_set
  #return dataset
  image_1 = dataset.image_1.values
  image_2 = dataset.image_2.values
  label = dataset.label.values
  features = {'image_1':image_1, 'image_2':image_2}
  dataset = tf.data.Dataset.from_tensor_slices((dict(features),label))
  return dataset

In [0]:
PATH_ORG = "drive/My Drive/Colab Notebooks/SigNet/signatures/signatures/full_org"
PATH_FORG = "drive/My Drive/Colab Notebooks/SigNet/signatures/signatures/full_forg"

def train_input():
  dataset = get_dataset(subset="train")
  dataset = dataset.shuffle(1814) #train_size = 2592*0.7
  dataset = dataset.map((lambda x,y:get_images(x,y)),num_parallel_calls=4)
  #dataset = dataset.map(standardize_images,num_parallel_calls=4)
  #dataset = dataset.map(preprocess_images,num_parallel_calls=4)
  dataset = dataset.repeat(100)#num_epochs
  dataset = dataset.batch(32)#batch_size
  dataset = dataset.prefetch(1)
  #print(dataset)
  return dataset

def val_input():
  dataset = get_dataset(subset="val")
  dataset = dataset.map((lambda x,y:get_images(x,y)),num_parallel_calls=4)
  dataset = dataset.batch(32)#batch_size
  dataset = dataset.prefetch(1)
  return dataset

def get_images(features,label):
  img_ = []
  for image_path in [features['image_1'],features['image_2']]:
    image_string = tf.read_file(image_path)
    img = tf.image.decode_jpeg(image_string, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize_images(img, [155, 220])#ref paper
    img_.append(img)
  feats = {'image_1':img_[0],'image_2':img_[1]}
  return feats, label

def standardize_images(image_1,image_2,label):
  std_img = []
  for img in [image_1,image_2]:
    mean = np.mean(img,axis=None)
    img = img - mean
    std = np.std(img)
    if(std == 0 or std == np.nan):
      std = 1
    standardized_img = img/std
    std_img.append(standardized_img)
  return std_img[0],std_img[1],label

In [0]:
import tensorflow as tf

In [0]:
def build_model(image,mode):
  inp = image
  
  conv_1 = tf.layers.conv2d(inputs=inp, filters=96, kernel_size=[11,11], padding="SAME", activation=tf.nn.relu)
  batch_norm_1 = tf.layers.batch_normalization(inputs=conv_1)
  max_pool_1 = tf.layers.max_pooling2d(inputs=batch_norm_1, pool_size=[3,3], strides=2)
  
  conv_2 = tf.layers.conv2d(inputs=max_pool_1, filters=256, kernel_size=[5,5], padding="SAME", activation=tf.nn.relu)
  batch_norm_2 = tf.layers.batch_normalization(inputs=conv_2)
  max_pool_2 = tf.layers.max_pooling2d(inputs=batch_norm_2, pool_size=[3,3], strides=2)
  
  dropout_1 = tf.layers.dropout(inputs=max_pool_2, rate=0.3)
  
  conv_3_a = tf.layers.conv2d(inputs=dropout_1, filters=384, kernel_size=[3,3], padding="SAME", activation=tf.nn.relu)
  conv_3_b = tf.layers.conv2d(inputs=conv_3_a, filters=256, kernel_size=[3,3], padding="SAME", activation=tf.nn.relu)
  max_pool_3 = tf.layers.max_pooling2d(inputs=conv_3_b, pool_size=[3,3], strides=2)
  
  dropout_2 = tf.layers.dropout(inputs=max_pool_3, rate=0.3)
  
  flat = tf.layers.flatten(inputs=dropout_2)
  
  fc_1 = tf.layers.dense(inputs=flat,units=1024,activation=tf.nn.relu)
  dropout_3 = tf.layers.dropout(inputs=fc_1, rate=0.5)
  fc_2 = tf.layers.dense(inputs=dropout_3,units=128,activation=tf.nn.relu)
  
  return fc_2

In [0]:
def get_loss(encoding_1,encoding_2,labels):
  d = tf.reduce_sum(tf.square(encoding_1-encoding_2),1)
  d_sqrt = tf.sqrt(d)
  labels = tf.cast(labels,dtype=tf.float32)
  loss = labels*tf.square(tf.maximum(0., 1 - d_sqrt)) + (1 - labels)*d
  loss = 0.5*tf.reduce_mean(loss)
  return loss

def get_pred(encoding_1,encoding_2):
  #print(tf.square(encoding_1-encoding_2))
  s = tf.reduce_sum(tf.square(encoding_1-encoding_2),1)
  #print(s)
  s_sqrt = tf.sqrt(s)
  #print(s_sqrt)
  th = tf.reduce_mean(s_sqrt,0,name="get_pred_mean")
  preds = tf.where(tf.greater_equal(s_sqrt,th))
  return preds

def build_fn(features,labels,mode,params):
  encoding_1 = build_model(features["image_1"],mode)
  encoding_2 = build_model(features["image_2"],mode)
  loss = get_loss(encoding_1,encoding_2,labels)
  
  predicted_label = get_pred(encoding_1,encoding_2)
  correct_prediction=tf.equal(labels, predicted_label)
  accuracy=tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  logging_hook = tf.train.LoggingTensorHook({"loss":loss,"accuracy":accuracy},every_n_iter=1)
  
  if mode == tf.estimator.ModeKeys.TRAIN:
  
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-2)#tf.train.AdadeltaOptimizer(learning_rate=0.01)
    train_step = optimizer.minimize(loss,global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_step, training_hooks = [logging_hook])
  
  accuracy_1 = tf.metrics.accuracy(labels=labels,predictions=predicted_label)
  eval_metric_op = {
      "accuracy" : accuracy_1
  }
  if mode == tf.estimator.ModeKeys.EVAL:
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_op)
  

In [0]:
estimator = tf.estimator.Estimator(model_fn=build_fn)
estimator.train(input_fn=(lambda :train_input()),steps=10)

W0706 08:05:29.432249 140484814280576 estimator.py:1811] Using temporary folder as model directory: /tmp/tmpgblyaiwo
W0706 08:05:29.435967 140484814280576 model_fn.py:630] Estimator's model_fn (<function build_fn at 0x7fc47cd789d8>) includes params argument, but params are not passed to Estimator.


In [0]:
estimator.evaluate(input_fn=(lambda :val_input()))

{'accuracy': 0.028277636, 'global_step': 10, 'loss': 0.248625}